# Automation for IB portfolios

Latest version: 2024-08-27  
Author: MvS

## Description

Notebook to automate calculation and setting of stop-loss in IB portfolios

## Result


### Warning: This notebook will place live orders


In [ ]:
from ib_async import *
import yfinance as yf

import pandas as pd
import numpy as np
from dotenv import dotenv_values

import logging
import datetime
import time

# log everything
if True:
    util.logToConsole(logging.DEBUG)

# ib_sysnc: notebook relevant
util.startLoop()

# read env vars
env_dict = dotenv_values("../.env")

ib = IB()
ib.connect(env_dict['IB_API_IP'], int(env_dict['IB_API_PORT']), clientId=int(env_dict['IB_API_KEY']))


### Get liquidation value of whole account

In [ ]:
logger = logging.getLogger(__name__)

liq_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

logger.info('Printing liquidation value:')
for counter, (acc, bal, curr) in enumerate(liq_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

### Unpack cash and stock balance

In [ ]:
cash_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'CashBalance' and v.currency != 'BASE']
stock_balance = [(v.account, v.value, v.currency) for v in ib.accountValues() if v.tag == 'StockMarketValue' and v.currency != 'BASE']

logger.info('Printing cash balance:')
for counter, (acc, bal, curr) in enumerate(cash_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

logger.info('Printing stock balance:')
for counter, (acc, bal, curr) in enumerate(stock_balance, 1):
    logger.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")
    
#ib.accountValues()

### Unpack position balance

In [ ]:
logger.info('Printing open positions:')
portfolio_pos = []

for counter, position in enumerate(ib.positions(), 1):

    contract = position.contract
    position_type = 'LONG' if position.position > 0.0 else 'SHORT'

    logger.info(f"{position.account}, {counter:0d}: {contract.conId}, {contract.secType},\
        {position_type}, {contract.symbol}, {contract.localSymbol},\
        {contract.exchange}, {contract.currency}, {position.position:8.2f},\
        {position.avgCost:8.2f}")

    portfolio_pos.append(
        [
            position.account,
            contract.conId,
            contract.secType,
            position_type,
            contract.symbol,
            contract.localSymbol,
            contract.exchange,
            contract.currency,
            position.position,
            position.avgCost,
        ]
    )

columns=['account', 'conId', 'secType', 'position_type', 'symbol', 'localSymbol', 'exchange', 'currency', 'position', 'avgCost']
portfolio_df = pd.DataFrame(portfolio_pos, columns=columns)

del portfolio_pos, contract, position_type, columns
portfolio_df

### Constructing the symbol for Yahoo Finance

The list of mappings needs to be updated as soon as new securities from different exchanges are traded

In [ ]:

exchange_map = {
    'IBIS': 'DE',
    'GETTEX2': 'MU',
    'FWB': 'DE',
}

def yf_symbol(x):
    ex = x['exchange']
    ex_sym = x['exchange_sym']
    sym = x['symbol']
    loc_sym = x['localSymbol']

    if not pd.isna(ex_sym):
        return f"{sym}.{ex_sym}"
    else:
        return f"{sym}"

portfolio_df['exchange_sym'] = portfolio_df['exchange'].map(exchange_map, na_action='ignore')
portfolio_df['yf_sym'] = portfolio_df.apply(yf_symbol, axis=1)
portfolio_df

### Download market data from [Yahoo Finance](https://finance.yahoo.com/)

In [ ]:
periods = 252
dt_end = datetime.datetime.today()

# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - plus considering non-trading days - yfinance returns only trading days
dt_data_start = dt_end - datetime.timedelta(days=365*1.2)

stock_data_list = []
stock_data_dict = {}

for symbol in portfolio_df['yf_sym'].to_list(): 
    time.sleep(1)
    try:
        # Grab sufficient stock data for averaging SMAs
        load_df = yf.download(
            symbol,
            start=dt_data_start.strftime('%Y-%m-%d'),
            end=dt_end.strftime('%Y-%m-%d'),
            progress=False,
        )

        assert load_df.shape[1] == 6 and load_df.shape[0] >= periods
        # data frame for processing
        stock_data_list.append((symbol, load_df))
        # last security price for portfolio
        stock_data_dict[symbol] = load_df['Adj Close'].iloc[-1]

    except AssertionError:
        print(f"Download failed for symbol {symbol}.  Skipping...")

portfolio_df['yf_adj_close'] = portfolio_df['yf_sym'].map(stock_data_dict, na_action='ignore')
portfolio_df

### Calculate isolated true range spikes

In [ ]:
from importlib import reload  # Python 3.4+
import utils.atr_utils as atr

reload(atr)

extrema_pos = []
for stock_symbol, stock_df in stock_data_list:
    result_extrema = [stock_symbol] + atr.calc_atr_spikes(stock_df, periods=periods)
    extrema_pos.append(result_extrema)

columns=['yf_sym', 'HR_gmean', 'HR_ex1', 'HR_ex2', 'HR_ex3', 'LR_gmean', 'LR_ex1', 'LR_ex2', 'LR_ex3']
extrema_df = pd.DataFrame(extrema_pos, columns=columns)

res_portfolio_df = pd.merge(portfolio_df, extrema_df, on="yf_sym")

del extrema_pos, columns, extrema_df

In [ ]:
def create_stop(x):
    # create stop-loss ranges
    perc = 1.000
    tr_scaler = 1.5

    last_price = x['yf_adj_close']
    pos_type = x['position_type']
    short = x['HR_gmean']
    long = x['LR_gmean']

    
    if pos_type =='LONG':
        # print(long * tr_scaler/last_price)
        return round((last_price - long * tr_scaler) / perc, 6)
    elif pos_type =='SHORT':
        # print(short * tr_scaler/last_price)
        return round((last_price + short * tr_scaler) * perc, 6)
    else:
        return 0.0

def calc_stop_perc(x):

    last_price = x['yf_adj_close']
    quant = x['position']
    stop = x['stop_price']
    avg_cost = x['avgCost']

    return (last_price - stop) * quant / abs(quant * avg_cost)

# Calculate Stop-price
res_portfolio_df['stop_price'] = res_portfolio_df.apply(create_stop, axis=1)

# Derive a limit shift
limit_factor = 0.01
res_portfolio_df['limit_delta'] = round(res_portfolio_df['stop_price'] * limit_factor, 6)

# Show the percentage of money at risk for this security
res_portfolio_df['stop_perc'] = round(res_portfolio_df.apply(calc_stop_perc, axis=1), 3)

res_portfolio_df

### For each position create a contract and a stop-loss order

- check tradability of contract
- find correct price increment and scale bid (stop/limit) prices 
- close previous stop/loss setting - preserving manual orders

In [ ]:
columns = ['secType', 'position_type', 'symbol', 'currency', 'exchange', 'position', 'avgCost', 'yf_adj_close', 'stop_price', 'limit_delta', 'stop_perc']

set_limit_portfolio = res_portfolio_df[columns].values.tolist()


res_portfolio_df[columns]

In [ ]:
# Get required increments for setting valid bid prices
def get_increments(symbol='NVDA', exchange='SMART', currency='USD'):
    # Set security
    port_stock = Stock(symbol=symbol, exchange=exchange, currency=currency)
    # Get tradability details
    details = ib.reqContractDetails(port_stock)
    assert len(details) == 1
    # Extract general exchanges and trading rules
    rules_list = [elem.strip() for elem in details[0].marketRuleIds.split(',')]
    exchange_list = [elem.strip() for elem in details[0].validExchanges.split(',')]
    rules_dict = dict(zip(exchange_list, rules_list))
    # Select specific rule set
    rules = ib.reqMarketRule(rules_dict[exchange])

    # Extract  rule set
    rules = [(rule.lowEdge, rule.increment) for rule in rules]
    rules.sort(key=lambda x: x[0], reverse = False)
    return rules


def cancel_previous_order(symbol='NVDA', currency='USD', order_type='STP LMT'):
    # Show all open orders, including chained/manual orders without id
    orders = ib.reqAllOpenOrders()

    # Cancel all stop limit orders - the chained orders survive this
    for order in orders:
        if (
                order.order.orderType == order_type 
                and order.contract.symbol == symbol
                and order.contract.currency == currency
                and order.order.orderId != 0
            ):
            trade = ib.cancelOrder(order.order)
            logger.info(f"Removing {order_type} order {order.order.orderId} for {symbol} trade")
            return trade
        else:
            return None

# Check for tradable contracts
for count, row in enumerate(set_limit_portfolio, 1):

    if row[0] == 'STK':
        symbol = row[2]
        currency = row[3]

        port_stock = Stock(symbol=symbol, exchange=row[4], currency=currency)
        details = ib.reqContractDetails(port_stock)

        assert len(details) == 1



# Set stop limits
for count, row in enumerate(set_limit_portfolio, 1):

    if row[0] == 'STK':

        exchange = 'SMART'
        symbol = row[2]
        currency = row[3]
        stop = row[8]
        quantity = abs(row[5])

        if row[1] == 'LONG':
            action = 'SELL'
            limit = stop - row[9]
        elif row[1] == 'SHORT':
            action = 'BUY'
            limit = stop + row[9]
        else:
            pass

        port_stock = Stock(symbol=symbol, exchange=exchange, currency=currency)
        ib.qualifyContracts(port_stock)
        increments = get_increments(symbol=symbol, exchange=exchange, currency=currency)
        
        # Get largest element that is still smaller than price bid
        increment = [inc for inc in increments if limit > inc[0]][-1][1]
        # Adjust prices to raster
        limit_round = int(limit / increment) * increment
        stop_round = int(stop / increment) * increment

        # Set up order
        stopLossLimitOrder = StopLimitOrder(action=action, totalQuantity=quantity, lmtPrice=limit_round, stopPrice=stop_round)
        stopLossLimitOrder.outsideRth = True
        stopLossLimitOrder.tif = 'GTC'

        current_price = row[7]

        # Log
        logger.info(f"{count:0d}: Setting {action} action for {row[2]} position of {quantity:.2f} units \
{symbol} at currently {current_price:.2f}, \
with adapted stop {stop_round:.5f} and limit {limit_round:.5f} at increment {increment:.5f}")
#with stop {stop:.5f} and limit {limit:.5f}, \
        
        # Find pre-existing orders and remove them, unless they are manual
        cancellation = cancel_previous_order(symbol=symbol, currency=currency, order_type='STP LMT')
        if cancellation is not None:
            ib.sleep(1)
            print(cancellation.orderStatus)
        
        # Place neworder
        limitTrade = ib.placeOrder(port_stock, stopLossLimitOrder)
        ib.sleep(1)
        assert limitTrade.orderStatus.status == 'PreSubmitted'
        print(limitTrade.orderStatus)


### Close Connection

In [ ]:
ib.disconnect()

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [ ]:
trade = ib.placeOrder(contract, order)

### List orders and cancel an order

In [ ]:
orders = ib.openOrders()
orders

In [ ]:
orders

In [ ]:
ib.cancelOrder(order=orders[1] )

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [ ]:
ib.sleep(1)
trade.log

``trade`` will also available from ``ib.trades()``:

In [ ]:
assert trade in ib.trades()

Likewise for ``order``:

In [ ]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [ ]:
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

``status`` will change from "PendingSubmit" to "Submitted":

In [ ]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [ ]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [ ]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

And now cancel it:

In [ ]:
ib.cancelOrder(limitOrder)

In [ ]:
limitTrade.log

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [ ]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

What are our positions?

In [ ]:
ib.positions()

What's the total of commissions paid today?

In [ ]:
sum(fill.commissionReport.commission for fill in ib.fills())

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [ ]:
order = MarketOrder('SELL', 20000)
ib.whatIfOrder(contract, order)

In [ ]:
ib.disconnect()